# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [67]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

keywords = ['python', 'алгоритм']
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
# добираемся до постов
articles = soup.find_all('article', class_= 'post post_preview')
# получаем ссылки
links = list(map(lambda x: x.find('a', class_= 'post__title_link').get('href'), articles))
# получаем текст preview
post_body = list(map(lambda x: x.find('div', class_= 'post__body post__body_crop').text.strip(), articles))

In [68]:
hub_pre = pd.DataFrame()
# итерация по ссылкам и preview
for link, post in zip(links, post_body):
    # проверка условия вхождения слов в preview
    if any([keyword in post.lower() for keyword in keywords]):
        # Дата публикации находится на полной странице. Для этого получаем страницу целиком.
        req_full = requests.get(link) 
        soup_full = BeautifulSoup(req_full.text, 'html.parser')
#         Вытаскиваем дату
        date = pd.to_datetime(soup_full.find('span', class_ = 'post__time').get('data-time_published')).date()
#     Получаем название
        title = soup_full.find('span', class_ = 'post__title-text').text
        row = {'Дата':date, 'Заголовок':title, 'Ссылка': link, }
        hub_pre = pd.concat([hub_pre, pd.DataFrame([row])])
hub_pre   


,Дата,Заголовок,Ссылка
0,2020-12-15,Radix sort — выжать всё,https://habr.com/ru/post/533206/


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [69]:
hub_full = pd.DataFrame()
for link in links:
    req_full = requests.get(link)
    soup_full = BeautifulSoup(req_full.text, 'html.parser')
    post = soup_full.find('div', class_= 'post__body post__body_full').text.strip()
    if any([keyword in post.lower() for keyword in keywords]): 
        date = pd.to_datetime(soup_full.find('span', class_ = 'post__time').get('data-time_published')).date()
        title = soup_full.find('span', class_ = 'post__title-text').text
        row = {'Дата':date, 'Заголовок':title, 'Ссылка': link, 'Текст_статьи': post}
        hub_full = pd.concat([hub_full, pd.DataFrame([row])])               
          
hub_full


,Дата,Заголовок,Ссылка,Текст_статьи
0,2020-12-15,Radix sort — выжать всё,https://habr.com/ru/post/533206/,Приветствую всех любителей алгоритмов. Хочу Ва...
0,2020-12-15,Ковыряемся внутри умной тепловой камеры с микр...,https://habr.com/ru/company/itelma/blog/533250/,Китайская компания Hikvision — один из ведущих...
0,2020-12-15,Как написать (игрушечную) JVM,https://habr.com/ru/company/timeweb/blog/533244/,Статья про KVM оказалась интересной для читате...
0,2020-12-15,Как далеки мы от беспроводного электричества?,https://habr.com/ru/company/skillfactory/blog/...,Привет Хабр! Я хочу рассказать тебе историю о ...
0,2020-12-15,Бенчмарки Apple M1 в реальной разработке,https://habr.com/ru/post/533232/,Я очень впечатлён результатами тестов Apple M1...
0,2020-12-15,Построение инфраструктуры распределенной трасс...,https://habr.com/ru/company/otus/blog/533234/,"Для будущих учащихся на курсе ""Highload Archit..."
0,2020-12-15,Head-of-Line Blocking в QUIC и HTTP/3: Подробн...,https://habr.com/ru/company/selectel/blog/532868/,"Как вы могли слышать, после четырех лет разраб..."


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [56]:
import requests
import pandas as pd
import time
import json

EMAIL = ['xxx@ya.ru', 'yyy@y.com']
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

res = pd.DataFrame({})
for email in EMAIL:
    data = {'emailAddresses': [email]}
    time.sleep(0.5)
    req = requests.post(URL, json = data, headers = {'Vaar-Version': '0', 'Vaar-Header-App-Product': 'hackcheck-web-avast'})
    df = pd.DataFrame(req.json()['breaches']).transpose()
    df['email'] = email  
    res = pd.concat([df, res])
res2 = res.loc[:,['email', 'publishDate', 'site', 'description']]
res2.columns = ['почта', 'дата утечки', 'источник утечки', 'описание утечки']
res2

,почта,дата утечки,источник утечки,описание утечки
17110,yyy@y.com,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
17670,yyy@y.com,2020-05-28T00:00:00Z,wishbone.io,"In January 2020, the online poll website Wishb..."
13094,yyy@y.com,2017-11-04T00:00:00Z,myheritage.com,"In October 2017, a customer database belonging..."
16768,yyy@y.com,2019-06-13T00:00:00Z,canva.com,"In May 2019, graphic-design site Canva's datab..."
41,yyy@y.com,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
2,yyy@y.com,2016-10-21T00:00:00Z,linkedin.com,"In 2012, online professional networking platfo..."
3587,yyy@y.com,2017-03-01T00:00:00Z,rayli.com.cn,"On an unconfirmed date, Chinese gossip site Ra..."
17009,yyy@y.com,2019-10-17T00:00:00Z,zynga.com,"In September 2019, the game developer Zynga wa..."
3,yyy@y.com,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
13662,yyy@y.com,2018-02-18T00:00:00Z,netlog.com,Netlog (formerly known as Facebox and Bingbox)...


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

In [6]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

NEWSFEED_REQUEST = 'https://api.vk.com/method/wall.get?'
TOKEN = '0'
VERSION = '5.103'
SLEEP = 0.33
GROUP = 'netology'
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'count': 50,
    'domain':GROUP,
}
res = requests.get(NEWSFEED_REQUEST, params)
df = pd.DataFrame(res.json()['response']['items'])[['date', 'text']]
df.date = pd.to_datetime(df.date, unit='s')
df

,date,text
0,2020-12-07 10:05:00,Наш предновогодний digital-марафон продолжаетс...
1,2020-12-13 13:18:00,🎁 Запустили страницу с подарками и скидками от...
2,2020-12-13 08:19:00,Чужой пример заразителен 😉\nИстории успеха кру...
3,2020-12-12 12:56:00,В новой афише мероприятий конференции по марке...
4,2020-12-12 08:04:01,"Как упростить работу с таблицами, улучшить диа..."
5,2020-12-11 14:18:00,"В эфире #пятничнаябеседка. Здесь мы обсуждаем,..."
6,2020-12-11 07:28:00,Поиграем со шрифтами?😅 \n\nРазбираемся с графи...
7,2020-12-10 14:48:00,Учёт доходов и расходов — первый шаг к устране...
8,2020-12-10 11:18:00,*партнёрский пост*\n\nПодводим итоги года с Le...
9,2020-12-10 07:19:00,"Будущее ближе, чем мы думаем 🤖 \n \nВ новом Da..."
